# Desafio P&D (Machine Learning) Intelivix

Em processamento e entendimento de linguagem natural, a análise de sentimento é uma das áreas que mais têm recebido atenção da comunidade científica. Os seus desafios encontram-se principalmente na identificação e tratamento adequado de sarcasmo, negação, ambiguidade linguística, etc. Este desafio consiste em classificar os trechos de textos opinativos sobre filmes presentes na base fornecida em 5 níveis de sentimento: negativo, um pouco negativo, neutro, um pouco positivo e positivo.

Sobre a entrega:

1. Deve-se escolher 3 diferentes algoritmos de classificação ou regressão. Deve-se utilizar apenas o arquivo `train.tsv` para criar as bases de treino, validação e teste, comparando os algoritmos com a base de teste e escolhendo o melhor, justificando a escolha.

2. Os códigos e o relatório devem ser entregues em um ipython notebook didático, o qual deve ser auto-suficiente para ser executado (assumindo que o computador a executar possua todas as ferramentas necessárias instaladas).

3. O relatório deve conter todas as tentativas para resolver o problema, como se estivesse contando a história da estrada percorrida para se chegar no resultado.

## 1. Introdução

Para dar início à resolução deste desafio, seguem os _imports_ de todos os recursos que serão utilizados, dentre eles a biblioteca **`pandas`** para manipulação dos dados e a **`scikit-learn`** para o uso de algoritmos de aprendizagem e avaliação dos resultados. 

In [1]:
import pandas as pd

from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict, train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

## 2. Coleta

Os dados foram fornecidos previamente e os que serão utilizados estão dispostos no arquivo `train.tsv`. Serão lidos e armazenados na variável `data` para que possam ser pré-processados. A base de dados é composta de 4 colunas, são elas: 

- **Id**: identificador único de cada registro (linha) do conjunto de dados.
- **IdSentenca**: id de cada opinião sobre filmes. Vários registros podem ter o mesmo `IdSentenca`, pois os textos opinativos são "quebrados" em partes menores, formando novos registros.
- **Texto**: o conteúdo do texto em si.
- **Sentimento**: a classificação do sentimento relacionado àquele texto, podendo assumir os valores 0 (negativo), 1 (pouco negativo), 2 (neutro), 3 (pouco positivo) ou 4 (positivo).

In [2]:
data = pd.read_csv("dados/train.tsv", sep="\t")
data.head()

,Id,IdSentenca,Texto,Sentimento
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


# 3. Pré-processamento

Pode-se notar abaixo que o conjunto de dados possui grande quantidade de registros, também possui grande quantidade de ruído visto que as sentenças são quebradas em partes menores até chegar em uma única palavra. Supondo que não seja possível inferir sentimento para algumas palavras soltas e sem contexto, nota-se alguma forma de inconsistência na base dados. Vejamos as informações abaixo:

In [3]:
print('O dataset contém {} registros, dentre os quais:'.format(data.shape[0]))

print('- {} são opiniões negativas'.format(data['Sentimento'].value_counts()[0]))
print('- {} são opiniões pouco negativas'.format(data['Sentimento'].value_counts()[1]))
print('- {} são opiniões neutras'.format(data['Sentimento'].value_counts()[2]))
print('- {} são opiniões pouco positivas'.format(data['Sentimento'].value_counts()[3]))
print('- {} são opiniões positivas'.format(data['Sentimento'].value_counts()[4]))

O dataset contém 156060 registros, dentre os quais:
- 7072 são opiniões negativas
- 27273 são opiniões pouco negativas
- 79582 são opiniões neutras
- 32927 são opiniões pouco positivas
- 9206 são opiniões positivas


Dito isto, faz-se necessário pensar em uma forma de limpar este _dataset_, de maneira a eliminar os dados ruidosos e manter as informações relevantes para o desenvolvimento da solução do problema. Então, surgiu a ideia de manter no _dataset_ apenas a sentença principal, mas não utilizar a classificação de sentimento atribuído a ela a priori, e sim obter e arrendodar a média da classificação de todas as sentenças que são partes dela, como pode ser visto a seguir:

In [4]:
texts = []
sentiments = []
sentences_id = []

for sentence_id in data['IdSentenca'].unique():
    for index, row in data[data['IdSentenca'] == sentence_id].iterrows():
        sentences_id.append(row['IdSentenca'])
        texts.append(row['Texto'])
        sentiments.append(data[data['IdSentenca'] == sentence_id]['Sentimento'].mean())
        break

new_data = pd.DataFrame(data={'IdSentenca': sentences_id, 'Texto': texts, 'Sentimento': sentiments})
new_data['Sentimento'] = new_data['Sentimento'].apply(lambda x: round(x))

Pode ser vista abaixo a nova configuração dos registros e classes da base de dados:

In [5]:
print('O dataset agora contém {} registros, dentre os quais:'.format(new_data.shape[0]))

print('- {} são opiniões negativas'.format(new_data['Sentimento'].value_counts()[0]))
print('- {} são opiniões pouco negativas'.format(new_data['Sentimento'].value_counts()[1]))
print('- {} são opiniões neutras'.format(new_data['Sentimento'].value_counts()[2]))
print('- {} são opiniões pouco positivas'.format(new_data['Sentimento'].value_counts()[3]))
print('- {} são opiniões positivas'.format(new_data['Sentimento'].value_counts()[4]))

O dataset agora contém 8529 registros, dentre os quais:
- 92 são opiniões negativas
- 1329 são opiniões pouco negativas
- 5078 são opiniões neutras
- 1877 são opiniões pouco positivas
- 153 são opiniões positivas


Tem-se agora um conjunto de dados mais enxuto, porém ainda contendo as informações relevantes. O próximo passo é manipulá-lo de forma que seja possível aplicar corretamente os algoritmos de classificação.

**OBS**: a fim de otimizar a remoção de ruídos da base de dados e os resultados de classificação, surgiu a ideia de incluir um método na etapa de pré-processamento para remover as `stopwords` dos textos opinativos, utilizando a `Natural Language Toolkit (NLTK)`, biblioteca Python. `Stopwords` são palavras consideradas "irrelevantes" em algum contexto de alguma linguagem (nesse caso, no inglês), como por exemplo: _does_, _are_, _not_ etc. A `NLTK` provê diversas `stopwords` que remetem a alguma ação negativa e, por esse motivo, a ideia de removê-las da base de dados caiu por terra, pois no contexto de análise de sentimentos é importante avaliar proposições negativas.   

## 4. Algoritmos de Classificação

Primeiramente, antes de utilizar algoritmos de aprendizagem, é preciso criar um _vectorizer_ que será responsável por transformar o texto bruto em notação de __[<font color='blue'>bag-of-words</font>](https://machinelearningmastery.com/gentle-introduction-bag-words-model/)__ (saco de palavras) e/ou __[<font color='blue'>tf-idf</font>](http://www.tfidf.com/)__ (_term frequency – inverse document frequency_ ou, em português, frequência do termo – inverso da frequência nos documentos) para que, dessa forma, possam ser aplicados classificadores. Neste caso, serão testados dois _vectorizers_ (__[<font color='blue'>CountVectorizer</font>](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)__ e __[<font color='blue'>TfidfVectorizer</font>](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)__), ambos transformam o texto bruto na notação `bag-of-words`, porém o segundo usa a `bag-of-words` para colocar o texto em sua representação `tf-idf`.

In [6]:
vectorizers = [CountVectorizer, TfidfVectorizer]

Feito isto, foram escolhidos para serem testados 3 algoritmos de aprendizagem: __[<font color='blue'>Naïve Bayes</font>](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html)__, __[<font color='blue'>Regressão Logística</font>](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)__ e __[<font color='blue'>Random Forest</font>](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)__.

In [7]:
learning_models = [MultinomialNB, RandomForestClassifier, LogisticRegression]

Serão feitas iterações entre estes vetorizadores e algoritmos de aprendizagem a fim de comparar o resultado final (acurácia) e obter a melhor combinação de ambos. Para isto, o conjunto de dados será dividido em 70% para treino e 30% para teste dos modelos.

In [8]:
accuracy_dict = {}

# dividir conjuntos de treino (70%) e teste (30%)
X_train, X_test, y_train, y_test = train_test_split(new_data['Texto'], 
                                                    new_data['Sentimento'], 
                                                    test_size=0.3,
                                                    random_state=101)

for vectorizer in vectorizers:
    vect = vectorizer(analyzer='word')
    train_txt = vect.fit_transform(X_train)
    test_txt = vect.transform(X_test)
    for learning_model in learning_models:
        model = learning_model()
        model.fit(train_txt, y_train)
        model.predict(test_txt)
        results = cross_val_predict(model, train_txt, y_train, cv=10)
        accuracy_dict[accuracy_score(y_train, results)] = [vectorizer, learning_model]

# acurácias obtidas para cada combinação vetorizador x classificador
import pprint
pprint.pprint(accuracy_dict) # pretty print

{0.60067001675041876: [<class 'sklearn.feature_extraction.text.TfidfVectorizer'>,
                       <class 'sklearn.naive_bayes.MultinomialNB'>],
 0.60837520938023448: [<class 'sklearn.feature_extraction.text.TfidfVectorizer'>,
                       <class 'sklearn.ensemble.forest.RandomForestClassifier'>],
 0.61122278056951429: [<class 'sklearn.feature_extraction.text.CountVectorizer'>,
                       <class 'sklearn.ensemble.forest.RandomForestClassifier'>],
 0.6194304857621441: [<class 'sklearn.feature_extraction.text.CountVectorizer'>,
                      <class 'sklearn.naive_bayes.MultinomialNB'>],
 0.63249581239530983: [<class 'sklearn.feature_extraction.text.TfidfVectorizer'>,
                       <class 'sklearn.linear_model.logistic.LogisticRegression'>],
 0.64857621440536017: [<class 'sklearn.feature_extraction.text.CountVectorizer'>,
                       <class 'sklearn.linear_model.logistic.LogisticRegression'>]}


In [9]:
# obtendo a melhor combinação vetorizador x classificador
accuracy_dict[sorted(accuracy_dict.keys(), reverse=True)[0]]

[sklearn.feature_extraction.text.CountVectorizer,
 sklearn.linear_model.logistic.LogisticRegression]

Como pode ser visto acima, a melhor combinação Vetorizador x Classificador é aplicar o `CountVectorizer` em conjunto com o modelo de Regressão Logística. Então, o fluxo será executado mais uma vez, a fim de que os resultados possam ser melhor analisados.

In [10]:
X_train, X_test, y_train, y_test = train_test_split(new_data['Texto'], 
                                                    new_data['Sentimento'], 
                                                    test_size=0.3,
                                                    random_state=101)

model = LogisticRegression()
vectorizer = CountVectorizer(analyzer='word')

train_texts = vectorizer.fit_transform(X_train)
model.fit(train_texts, y_train)
test_texts = vectorizer.transform(X_test)
model.predict(test_texts)

array([2, 2, 2, ..., 2, 3, 2], dtype=int64)

## 5. Resultados

Após executar novamente o modelo de aprendizagem, os resultados serão analisados utilizando a técnica de _k-Fold Cross Validation_ (com `k=10`). Desta forma, foram calculadas as seguintes métricas: acurácia e matriz de confusão.

In [11]:
results = cross_val_predict(model, train_texts, y_train, cv=10) # 10-fold cross validation
accuracy = accuracy_score(y_train, results)

print('O modelo tem uma acurácia de {:.3f}%'.format(accuracy*100))

O modelo tem uma acurácia de 64.858%


In [12]:
print(pd.crosstab(y_train, results, rownames=['Real'], colnames=['Predito'], margins=True))

Predito  0    1     2     3  4   All
Real                                
0        0   31    36     2  0    69
1        1  253   643    22  0   919
2        0  189  3039   339  1  3568
3        0    9   719   576  3  1307
4        0    1    33    69  4   107
All      1  483  4470  1008  8  5970


## 6. Conclusões

Primeiramente, como já foi concluído há alguns passos atrás, o algoritmo de aprendizagem que obteve melhor acurácia foi a Regressão Logística. No entanto, a acurácia dos outros dois algoritmos testados também foi relativamente alta, ficando abaixo da Regressão Logística por apenas 1 a 4%. Além disso, nota-se que na matriz de confusão acima, a maior quantidade de números (predições) encontra-se mais no centro e na diagonal principal da matriz. Isto pode ser visto como positivo, pois mostra que, quanddo há erros de predição, não são erros "gritantes" e sim erros leves, como por exemplo: confundir opinião pouco negativa ou pouco positiva com neutra e vice-versa.